# Run macro file 

This notebook shows how load an ImageJ macrofile linked to a dataset and analyze all the images contained in that dataset with the macro.
This notebook assumes that a macro file has been linked to the dataset. If more than one macro is linked, the first one found will be applied.

Fiji has been installed with few other plugins. For more information, check the Dockerfile.

In [1]:
//Add dependencies to the classpath
%classpath add jar /opt/java-apps/Fiji.app/jars/*
%classpath add jar /opt/java-apps/Fiji.app/plugins/*
%classpath add jar /opt/java-apps/Fiji.app/plugins/OMERO.imagej-5.5.6/*

### Description

The following section shows
 * how to load a macro file linked to a dataset
 * how to analyze all the images in the dataset using the macro
 

In [2]:
g = new EasyForm("Enter credentials and continue to the next cell. Do not re-run this cell.")
g.addTextField("Server").onInit({g['Server'] = "workshop.openmicroscopy.org"})
g.addTextField("UserName")
g.addPasswordField("Password")
g


### Connect to OMERO

In [4]:
import omero.gateway.Gateway
import omero.gateway.LoginCredentials
import omero.log.SimpleLogger

// Method to connect to OMERO
def connect_to_omero() {
    "Connect to OMERO"

    credentials = new LoginCredentials()
    credentials.getServer().setHostname(g['Server'])
    credentials.getServer().setPort(4064)
    credentials.getUser().setUsername(g['UserName'].trim())
    credentials.getUser().setPassword(g['Password'].trim())
    simpleLogger = new SimpleLogger()
    gateway = new Gateway(simpleLogger)
    gateway.connect(credentials)
    return gateway

}

// Connect to OMERO
println "connecting..."
gateway = connect_to_omero()
println "connected..."


connecting...
connected...


null

In [5]:
ga = new EasyForm("Select the Dataset to analyze. Do not re-run this cell.")
ga.addTextField("DatasetID")
ga

In [6]:
import omero.gateway.SecurityContext
import omero.gateway.facility.BrowseFacility
import omero.gateway.facility.MetadataFacility
import omero.gateway.model.FileAnnotationData
import omero.gateway.model.DatasetData
import omero.model.OriginalFile

//Dataset to analyze Convert the String as long
dataset_id = ga['DatasetID'].toLong()

// Retrieve the macros linked to the dataset
user = gateway.getLoggedInUser()
ctx = new SecurityContext(user.getGroupId())
svc = gateway.getFacility(MetadataFacility)
//Find the dataset
browse = gateway.getFacility(BrowseFacility)
object = browse.findIObject(ctx, "omero.model.Dataset", dataset_id)
dataset = new DatasetData(object);
users = new ArrayList(1)
//users.add(new Long(user.getId()))
types = new ArrayList(1)
types.add(FileAnnotationData.class)
annotations = svc.getAnnotations(ctx, dataset, types, users)
macros = new HashMap()
macroNames = new HashMap()
querySvc = gateway.getQueryService(ctx)
annotations.each { fa ->
    of = (OriginalFile) querySvc.get("omero.model.OriginalFile", fa.getFileID());
    name = of.getName().getValue()
    // load the first annotation with ijm
    if (name.endsWith(".ijm")) {
        macros.put(fa.getFileID(), of)
        macroNames.put(name, fa.getFileID())
    }
}

if (macros.size) {
    println "No macros linked to the dataset"
    return
}
gm = new EasyForm("Select the macro to use. Do not re-run this cell.")
gm.addComboBox("Name", new ArrayList(macroNames.keySet()))
gm

Depending on the macros run, new images, ROIs or results will be created. Select the output you wish to save back to OMERO.

In [13]:
gr = new EasyForm("Select the output to save. Do not re-run this cell.")
options = ["Image", "ROIs", "Results"]
gr.addCheckBox("Image")
gr.addCheckBox("ROIs")
gr.addCheckBox("Results")
gr

### Run the macro

In [16]:
import java.util.ArrayList
import java.lang.Math
import java.lang.StringBuffer
import java.nio.ByteBuffer
import java.io.File
import java.io.PrintWriter

import java.nio.file.Files

import omero.gateway.SecurityContext
import omero.gateway.facility.BrowseFacility
import omero.gateway.facility.DataManagerFacility
import omero.gateway.facility.ROIFacility
import omero.gateway.model.FileAnnotationData
import omero.gateway.model.DatasetData
import omero.gateway.model.ImageData

import omero.model.DatasetI
import omero.model.ImageI
import omero.log.SimpleLogger
import omero.model.ChecksumAlgorithmI
import omero.model.FileAnnotationI
import omero.model.OriginalFile
import omero.model.OriginalFileI
import omero.model.enums.ChecksumAlgorithmSHA1160

import static omero.rtypes.rlong
import static omero.rtypes.rstring


//Internal class of OMERO.insight
import org.openmicroscopy.shoola.util.roi.io.ROIReader
import org.openmicroscopy.shoola.env.data.model.FileObject

// Dependencies to import data
import ome.formats.importer.ImportConfig
import ome.formats.importer.OMEROWrapper
import ome.formats.importer.ImportLibrary
import ome.formats.importer.ImportCandidates
import ome.formats.importer.cli.ErrorHandler
import ome.formats.importer.cli.LoggingImportMonitor

import loci.formats.in.DefaultMetadataOptions
import loci.formats.in.MetadataLevel

import ij.IJ
import ij.plugin.frame.RoiManager
import ij.measure.ResultsTable

//Connection information
HOST = g['Server']
USERNAME = g['UserName']
PASSWORD = g['Password']

PORT = 4064
//size of the chunk to load
INC = 262144
//Dataset to analyze Convert the String as long
dataset_id = ga['DatasetID'].toLong()

//Name of the macro
macro_name = gm['Name']
save_image = gr['Image']
save_rois = gr['ROIs']
save_results = gr['Results']

// Helper methods
// Load the image contained in the specified dataset
def get_images(gateway, dataset_id) {
    "List all image's ids contained in a Dataset"
    
    user = gateway.getLoggedInUser()
    ctx = new SecurityContext(user.getGroupId())
    browse = gateway.getFacility(BrowseFacility)
    ids = new ArrayList(1)
    ids.add(new Long(dataset_id))
    return browse.getImagesForDatasets(ctx, ids)
}

// Open the image using Bio-Formats
def open_image_plus(HOST, USERNAME, PASSWORD, PORT, group_id, image_id) {
    "Open the image using the Bio-Formats Importer"

    StringBuilder options = new StringBuilder()
    options.append("location=[OMERO] open=[omero:server=")
    options.append(g['Server'])
    options.append("\nuser=")
    options.append(g['UserName'].trim())
    options.append("\nport=")
    options.append(4064)
    options.append("\npass=")
    options.append(g['Password'].trim())
    options.append("\ngroupID=")
    options.append(group_id)
    options.append("\niid=")
    options.append(image_id)
    options.append("] ")
    options.append("windowless=true view=Hyperstack ")
    IJ.runPlugIn("loci.plugins.LociImporter", options.toString())
}

// Load the macro file linked to the dataset
def loag_macro_file(gateway, macro_name) {
    
    
    fileID = macroNames.get(macro_name)
    of = macros.get(fileID)

    user = gateway.getLoggedInUser()
    ctx = new SecurityContext(user.getGroupId())
    store = gateway.getRawFileService(ctx);
    file = File.createTempFile(name, ".ijm")
    stream = new FileOutputStream(file)
    store.setFileId(fileID);
    offset = 0;
    size = of.getSize().getValue();
    try {
        for (offset = 0; (offset+INC) < size;) {
            stream.write(store.read(offset, INC));
            offset += INC;
        }
    } finally {
        stream.write(store.read(offset, (int) (size-offset)));
        stream.close();
    }
    return file
}

// Upload the CSV file to OMERO
def upload_csv_to_omero(ctx, file, image_id) {
    "Upload the CSV file and attach it to the specified project"
    svc = gateway.getFacility(DataManagerFacility)

    file_size = file.length()
    original_file = new OriginalFileI()
    original_file.setName(rstring(file.getName()))
    original_file.setPath(rstring(file.getAbsolutePath()))
    original_file.setSize(rlong(file_size))
    checksum_algorithm = new ChecksumAlgorithmI()
    checksum_algorithm.setValue(rstring(ChecksumAlgorithmSHA1160.value))
    original_file.setHasher(checksum_algorithm)
    original_file.setMimetype(rstring("text/csv"))
    original_file = svc.saveAndReturnObject(ctx, original_file)
    store = gateway.getRawFileService(ctx)

    // Open file and read stream
    INC = 262144
    pos = 0
    buf = new byte[INC]
    ByteBuffer bbuf = null
    stream = null
    try {
        store.setFileId(original_file.getId().getValue())
        stream = new FileInputStream(file)
        while ((rlen = stream.read(buf)) > 0) {
            store.write(buf, pos, rlen)
            pos += rlen
            bbuf = ByteBuffer.wrap(buf)
            bbuf.limit(rlen)
        }
        original_file = store.save()
    } finally {
        if (stream != null) {
            stream.close()
        }
        store.close()
    }
    // create the file annotation
    namespace = "training.demo"
    fa = new FileAnnotationI()
    fa.setFile(original_file)
    fa.setNs(rstring(namespace))

    data_object = new ImageData(new ImageI(image_id, false)) 
    svc.attachAnnotation(ctx, new FileAnnotationData(fa), data_object)
}

//Upload the generated image
def upload_image(paths, gateway, id) {
    "Upload an image to OMERO"

    user = gateway.getLoggedInUser()
    sessionKey = gateway.getSessionId(user)

    config = new ImportConfig()
    config.debug.set('false')
    config.hostname.set(HOST)
    config.sessionKey.set(sessionKey)
    dataset = find_dataset(gateway, id)

    store = config.createStore()
    reader = new OMEROWrapper(config)

    library = new ImportLibrary(store, reader)
    error_handler = new ErrorHandler(config)

    library.addObserver(new LoggingImportMonitor())
    candidates = new ImportCandidates(reader, paths, error_handler)
    containers = candidates.getContainers()
    containers.each() { c ->
        c.setTarget(dataset)
    }
    reader.setMetadataOptions(new DefaultMetadataOptions(MetadataLevel.ALL))
    return library.importCandidates(config, candidates)
}

// Find the dataset matching the specified ID
def find_dataset(gateway, dataset_id) {
    "Load the Dataset"
    browse = gateway.getFacility(BrowseFacility)
    user = gateway.getLoggedInUser()
    ctx = new SecurityContext(user.getGroupId())
    return browse.findIObject(ctx, "omero.model.Dataset", dataset_id)
}

// Main part of the script
exp = gateway.getLoggedInUser()
group_id = exp.getGroupId()
exp_id = exp.getId()

// Load all the images contained in the dataset
println "loading images..."
images = get_images(gateway, dataset_id)

// Load the file annotations linked to the dataset.
println "loading macro..."
macro_file = loag_macro_file(gateway, macro_name)

//Apply the macro to all the images in the dataset
println "analyzing images..."

tmp_dir = Files.createTempDirectory("Fiji_csv")

images.each { img ->
    // if target_user ~= None:
    // Switch context to target user and open omeroImage as ImagePlus object
    img_id = img.getId()
    
    imp = open_image_plus(HOST, USERNAME, PASSWORD, PORT, String.valueOf(group_id), String.valueOf(img_id))
    println "Image opened: "+img_id
    imp = IJ.getImage()
    file_object = new FileObject(imp)
    println "Running macro " + macro_name + " on image: "+img_id
    IJ.runMacroFile(macro_file.getAbsolutePath())

    // Save the ROIs
    reader = new ROIReader()
    roi_list = reader.readImageJROIFromSources(img_id, imp)
    if (roi_list != null && roi_list.size() > 0 && save_rois) {
        println "saving ROIs"
        roi_facility = gateway.getFacility(ROIFacility)
        result = roi_facility.saveROIs(ctx, img_id, exp_id, roi_list)
    }
    
    path = tmp_dir.resolve(img.getName()+"_results.csv")
    file_path = Files.createFile(path)
    file = new File(file_path.toString())
    //Save the results
    value = reader.readResults(file)
    if (value && save_results) { //some results to upload
        println "saving Results as CSV"
        upload_csv_to_omero(ctx, file, img_id)
    }
    file.delete()
    
    // Check if the macro created a new image
    if (file_object.isNewImage() && save_image) {
        file_to_import = file_object.getFileToImport()
        println "importing..."
        str2d = new String[1]
        str2d[0] = file_to_import.getAbsolutePath()
        success = upload_image(str2d, gateway, dataset_id)
        println "imported"
    }
    imp.changes = false     /// Prevent "Save Changes?" dialog
    imp.close()
    //clear the rois
    manager = ij.plugin.frame.RoiManager.getInstance()
    if (manager != null) {
        manager.reset()
    }
    // clear the table
    manager = ij.plugin.filter.Analyzer.getResultsTable()
    if (manager != null) {
        manager.reset()
    }
}

// delete the local copy of the temporary file and directory
dir = new File(tmp_dir.toString())
entries = dir.listFiles()
for (i = 0; i < entries.length; i++) {
    entries[i].delete()
}
dir.delete()

// Close the connection
gateway.disconnect()
//Delete macrofile
macro_file.delete()
println "processing done"

loading images...
loading macro...
analyzing images...
Image opened: 55458
Running macro crop.ijm on image: 55458
Image opened: 9542
Running macro crop.ijm on image: 9542
processing done


null